In [1]:
%pip install -q langchain-tavily


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

True

In [3]:
from langchain_tavily import TavilySearch   

search_tool = TavilySearch(
    tavily_api_key="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",
    max_results=5,
    topic="general",
    search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [4]:
pprint(search_tool.invoke({'query': '강남 신세계에서 밥먹을만한 곳 알려줘'}))

{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': '강남 신세계에서 밥먹을만한 곳 알려줘',
 'request_id': '06b79849-108b-4b7e-82f3-0662f98d052e',
 'response_time': 0.95,
 'results': [{'content': '강남신세계 혼밥맛집 (106곳) · 1. 소녀방앗간 서울고속터미널점 · 2. 타코벨 '
                         '서울고속버스터미널점 · 3. 소이연남 파미에스테이션점 · 4. 갓덴스시 센트럴시티점 · 5. '
                         '101번지',
              'raw_content': None,
              'score': 0.7304634,
              'title': "'강남신세계 혼밥' 맛집 빅데이터 추천순위 Top100",
              'url': 'https://www.diningcode.com/list.dc?query=%EA%B0%95%EB%82%A8%EC%8B%A0%EC%84%B8%EA%B3%84%20%ED%98%BC%EB%B0%A5'},
             {'content': '1. 더 마고 그릴 신세계강남 · 2. 모던눌랑 강남고속터미널 · 3. 에토레 파미에스테이션 '
                         '· 4. 자주테이블 신세계강남 · 5. 쓰리버즈 센트럴시티점 · 6. 더 키친 일뽀르노 신세계',
              'raw_content': None,
              'score': 0.6907474,
              'title': "'강남신세계백화점 레스토랑' 맛집 빅데이터 추천순위 Top10",
              'url': 'https://www.diningcode.com/list.dc?query=%EA%B0%95%EB%82%A8%EC%8

In [5]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

llm = ChatOpenAI(model='gpt-4.1-nano')

prompt = PromptTemplate(
    input_variables=['query', 'search_results'],
    template="""넌 웹 검색결과를 요약해서 사용자의 질문에 맞게 좋은 답을 해주는 챗봇이야.

    질문: {query}
    검색결과: {search_results}
    질문에 맞는 답을 검색결과 바탕으로 간단하게 알려줘
    """
)

chain = (
    {
        'query': RunnablePassthrough(),  # 사용자 입력 그대로 이 자리에 들어옴
        'search_results': search_tool | RunnableLambda(
            lambda x: '\n'.join(
                [f'-{r['title']} ({r['content']})' for r in x['results']]
            )
        )
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('강남 신세계에서 점심 먹기 좋은곳')

"강남 신세계백화점에서 점심 식사하기 좋은 곳으로는 맛집 추천 순위에 오른 '101번지남산돈까스'와 '한우리 센트럴시티점'이 있습니다. 특히 한우 소고기국수전골을 즐기실 수 있는 한우리도 인기입니다. 또한, 신세계백화점 내에는 다양한 레스토랑과 푸드코트가 있어 편리하게 식사하실 수 있어요."

In [6]:
dumb_chain = prompt | llm | StrOutputParser()

dumb_chain.invoke({'query': '강남 신세계에서 점심먹기 좋은곳', 'search_results': ''})

'강남 신세계 주변에서는 다양한 점심 맛집이 있습니다. 편안한 분위기의 한식당이나 한우 전문점, 퓨전 레스토랑 등 선택의 폭이 넓어요. 특히 신세계백화점 내부 또는 근처에 있는 레스토랑들을 이용하면 편리하게 점심을 즐기실 수 있습니다.'

In [7]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from datetime import datetime

today = datetime.today().strftime('%D')

search_tool = TavilySearch(
    tavily_api_key="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",
    max_results=5,
    topic='general'
)

prompt = ChatPromptTemplate.from_messages([
    ('system', f'너는 훌륭한 어시스턴트야. 반드시 웹 검색도구를 사용해서 정보를 얻어. 최대한 답을 잘 해보자. 오늘은 {today}야.'),
    MessagesPlaceholder(variable_name='messages'),
    MessagesPlaceholder(variable_name='agent_scratchpad')  # 도구(검색) 호출때 필요함
])

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool],
    prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=[search_tool], verbose=True)

agent_executor.invoke({'messages': [{'role': 'human', 'content': '오늘 날씨에 맞는 음악 추천해줘'}]})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '오늘 09/25의 날씨', 'search_depth': 'basic'}`


{'query': '오늘 09/25의 날씨', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.kmib.co.kr/article/view.asp?arcid=0017506095', 'title': '오늘의 날씨 - 오후 (2022년 09월 25일) - 국민일보', 'content': '9월 25일 일요일, 오후 날씨입니다. 이 시간 현재 대부분 지방이 맑은 날씨를 보이고 있고, 호남·경북 지방은 구름이 많으며, 경남·제주 지방은 흐린 날씨', 'score': 0.88822687, 'raw_content': None}, {'url': 'https://news.nate.com/view/20250903n17432', 'title': '오늘의 날씨 (25. 09. 03. 10시) - 네이트 뉴스', 'content': "공기는 깨끗합니다. 미세먼지 농도는 '좋음' 단계를 보이겠습니다. 낮 최고기온은 서울과 전주 33도, 대전과 대구는 32도까지 오르겠고요. 동풍이 부는", 'score': 0.7738498, 'raw_content': None}, {'url': 'https://v.daum.net/v/20250901120329168', 'title': '오늘의 날씨 (25. 09. 01. 10시) - Daum', 'content': '# KTV **음성재생 설정** **번역*beta*** Translated by kaka i **번역중** Now in translation **글자크기 설정** 파란원을 좌우로 움직이시면 글자크기가 변경 됩니다. (예시) 가장 빠른 뉴스가 있고 다양한 정보, 쌍방향 소통이 숨쉬는 다음뉴스를 만나보세요.

{'messages': [{'role': 'human', 'content': '오늘 날씨에 맞는 음악 추천해줘'}],
 'output': '오늘의 날씨는 전국적으로 맑거나 구름이 약간 있으나 전반적으로 쾌적한 상태입니다. 특히, 낮 기온이 높아져서 따뜻한 날씨가 예상됩니다.\n\n이런 날씨에 어울리는 음악은 활기차면서도 기분 좋게 느껴지는 곡들이 좋아요. 예를 들어, 밝고 경쾌한 팝, 인디 음악, 혹은 여유로운 재즈 음악이 적합하겠습니다. \n\n여기 추천하는 곡 몇 가지입니다:\n1. "Happy" - Pharrell Williams\n2. "Walking on Sunshine" - Katrina and the Waves\n3. "Good as Hell" - Lizzo\n4. "Can’t Stop the Feeling!" - Justin Timberlake\n5. "Banana Pancakes" - Jack Johnson\n\n이 음악들이 오늘의 상쾌한 날씨와 잘 어울려서 기분 좋은 하루 보내시길 바랍니다!'}